In [ ]:
import seaborn as sns


# Расчет базового значения NPC для вывода его в отн. ед.
PV_cost_c=1
BESS_cost_c=1
npv, npc, lcoe, capex, opex = model.NPV1(30, total_cost, load['load'].sum(), 350*PV_cost_c, 1000, 200*BESS_cost_c, 10, 50, 3, 0.07)          # Здесь функция расчитывающая lcoe и npc
npc_base = npc


# Первая строка значений для датасета
PV_cost_c=0.3
BESS_cost_c=0.3
npv, npc, lcoe, capex, opex = model.NPV1(30, total_cost, load['load'].sum(), 350*PV_cost_c, 1000, 200*BESS_cost_c, 10, 50, 3, 0.07)          # Здесь функция расчитывающая lcoe и npc



# Датасет для LCOE, со значениями с пропусками "через 3"
df_lcoe = pd.DataFrame(                
    {
        "lcoe": [lcoe*1000],
        "Относительная удельная ст. PV": [PV_cost_c],
        "Относительная удельная ст. BESS": [BESS_cost_c],
    }
)


#Цикл с расчетом каждого 3-го значения lcoe
t=0
while PV_cost_c <= 1:
    BESS_cost_c=0.3
    while BESS_cost_c <= 1:
        if t >= 4: 
            npv, npc, lcoe, capex, opex = model.NPV1(30, total_cost, load['load'].sum(), 350*PV_cost_c, 1000, 200*BESS_cost_c, 10, 50, 3, 0.07)          # Здесь функция расчитывающая lcoe и npc
            df_lcoe.loc[ len(df_lcoe.index )] = [lcoe*1000, PV_cost_c, BESS_cost_c]
            t=0   
        BESS_cost_c += 0.05
        BESS_cost_c = round(BESS_cost_c, 2)
        t += 1
        t = round(t, 2)
    PV_cost_c += 0.05    
    PV_cost_c = round(PV_cost_c, 2)

# Формирование датасета в вид, пригодный для функции heatmap
summary_lcoe = pd.pivot_table(data=df_lcoe,index='Относительная удельная ст. PV',columns='Относительная удельная ст. BESS',values='lcoe')



PV_cost_c=0.3
BESS_cost_c=0.3
npv, npc, lcoe, capex, opex = model.NPV1(30, total_cost, load['load'].sum(), 350*PV_cost_c, 1000, 200*BESS_cost_c, 10, 50, 3, 0.07)          # Здесь функция расчитывающая lcoe и npc


# Датасет для NPC, без пропусков значений
df_npc = pd.DataFrame(                
    {
        "npc": [npc/npc_base],
        "Относительная удельная ст. PV": [PV_cost_c],
        "Относительная удельная ст. BESS": [BESS_cost_c],
    }
)

#Цикл с расчетом всех значений npc
while PV_cost_c <= 1:
    BESS_cost_c=0.3
    while BESS_cost_c <= 1:
        npv, npc, lcoe, capex, opex = model.NPV1(30, total_cost, load['load'].sum(), 350*PV_cost_c, 1000, 200*BESS_cost_c, 10, 50, 3, 0.07)          # Здесь функция расчитывающая lcoe и npc
        df_npc.loc[ len(df_npc.index )] = [npc/npc_base, PV_cost_c, BESS_cost_c]   
        BESS_cost_c += 0.05
        BESS_cost_c = round(BESS_cost_c, 2)
    PV_cost_c += 0.05    
    PV_cost_c = round(PV_cost_c, 2)

# Второй датасет для heatmap
summary_npc = pd.pivot_table(data=df_npc,index='Относительная удельная ст. PV',columns='Относительная удельная ст. BESS',values='npc')

# Отрисовка графика по двум датасетам
fig, ax = plt.subplots(figsize=(16, 9), dpi=200)
ax.text(15.6, 15.6, "NPC, отн. ед.", fontdict=None)
A = sns.heatmap(summary_lcoe, annot=True, cbar=False, cmap='Greys', vmin=0, vmax=10000)  # Здесь график с цифрами "каждое N-ое значение" (lcoe)
B = sns.heatmap(summary_npc, cmap='coolwarm')                           # График с цветами без цифр (npc)
